In [20]:
import os
import sys
import pandas as pd
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go

In [21]:
notebook_directory = Path().resolve().parent
sys.path.insert(0,str(notebook_directory.parent))

from experiment.helper_functions import load_dict

In [22]:
saved_path=r"D:/finaldraft/ExplainingWilderness/experiment/Notebooks/temp"
filename="_evaluation_metric_dict.pkl"
modeltype="resnet18"
evaluation_metric=load_dict(os.path.join(saved_path,modeltype+filename))

In [23]:
print(evaluation_metric)

{'IntergratedGradients': {'Robustness': 1.409381634183228, 'Faithfulness': 0.102183750136015, 'Complexity': 0.5607855494787594, 'Randomisation': 0.9389847326975578}, 'GradientShap': {'Robustness': 1.456339622940868, 'Faithfulness': 0.08903637309467224, 'Complexity': 0.5550686898688847, 'Randomisation': 0.7393987459994389}, 'GuidedGradCam': {'Robustness': 1.281200789194554, 'Faithfulness': 0.04777327427486408, 'Complexity': 0.7210471037631223, 'Randomisation': 0.7955206880423944}, 'LRP': {'Robustness': 24.94312889687717, 'Faithfulness': 0.1515888389167987, 'Complexity': 0.5492273203079003, 'Randomisation': 0.987188370056814}, 'CRP': {'Robustness': 4.835530694387853, 'Faithfulness': 0.12057726967603517, 'Complexity': 0.5524042031634515, 'Randomisation': 0.9999980712863398}, 'Occulsion': {'Robustness': 1.1789262627949937, 'Faithfulness': 0.09922280747262471, 'Complexity': 0.4032791718354568, 'Randomisation': 0.9134619564212294}}


In [24]:
def hex_to_rgba(hex_color, alpha):
    hex_color = hex_color.lstrip('#')
    hlen = len(hex_color)
    return 'rgba(' + ', '.join(str(int(hex_color[i:i+hlen//3], 16)) for i in range(0, hlen, hlen//3)) + f', {alpha})'

In [25]:
df = pd.DataFrame.from_dict(evaluation_metric)
df = df.T.abs()
print(df)

                      Robustness  Faithfulness  Complexity  Randomisation
IntergratedGradients    1.409382      0.102184    0.560786       0.938985
GradientShap            1.456340      0.089036    0.555069       0.739399
GuidedGradCam           1.281201      0.047773    0.721047       0.795521
LRP                    24.943129      0.151589    0.549227       0.987188
CRP                     4.835531      0.120577    0.552404       0.999998
Occulsion               1.178926      0.099223    0.403279       0.913462


In [26]:
# Take inverse ranking for Robustness, since lower is better.
df_normalised_rank = df.copy()
df_normalised_rank['Robustness'] = df['Robustness'].rank(ascending=True)
df_normalised_rank['Faithfulness'] = df['Faithfulness'].rank(ascending=False)
df_normalised_rank['Complexity'] = df['Complexity'].rank(ascending=True)
df_normalised_rank['Randomisation'] = df['Randomisation'].rank(ascending=False)
print(df_normalised_rank)

                      Robustness  Faithfulness  Complexity  Randomisation
IntergratedGradients         3.0           3.0         5.0            3.0
GradientShap                 4.0           5.0         4.0            6.0
GuidedGradCam                2.0           6.0         6.0            5.0
LRP                          6.0           1.0         2.0            2.0
CRP                          5.0           2.0         3.0            1.0
Occulsion                    1.0           4.0         1.0            4.0


In [27]:
# Performing dynamic range adjustment based on the data in the DataFrame
colors = px.colors.qualitative.Plotly
# Calculate the maximum value in the DataFrame for range adjustment
max_range = df_normalised_rank.max().max()
min_range = df_normalised_rank.min().min() if df_normalised_rank.min().min() < 0 else 0
markers = ['circle', 'square', 'diamond', 'cross', 'x', 'triangle-up']  # Different marker types

# Update the range dynamically based on the data
fig = go.Figure()

for i,(table,color,marker) in enumerate(zip(df_normalised_rank.iterrows(), colors, markers)):
    index,row=table
    r_values = list(row) + [row[0]]
    theta_values = list(df_normalised_rank.columns) + [df_normalised_rank.columns[0]]
    fig.add_trace(go.Scatterpolar(
        r=r_values,
        theta=theta_values,
        marker=dict(color=color, symbol=marker, size=10),
        line=dict(color=color, width=3),
        fill='toself',
        fillcolor=hex_to_rgba(color,0.05),
        name=index
    ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[min_range, max_range + 0.1 * max_range],  # Adding a buffer to the max range
            color='grey',
            gridcolor='grey',  # Color of the radial grid lines
            linecolor='grey'
        ),
        angularaxis=dict(
            gridcolor='white',  # Color of the angular grid lines
            linecolor='white'  # Color of the angular axis line
        )
        ),
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.25,
        xanchor="center",
        x=0.5
    ),
    width=700,
    height=500,
    paper_bgcolor='black',  # Set the background color to black
    font=dict(color='white'),
    title=dict(
        text="Quantitative Analysis of " +modeltype+ " Model Explanations",  # Replace with your desired title
        x=0.5,  # Title alignment (0.5 for center alignment)
        xanchor='center',  # Anchor the title at the center
        font=dict(
            size=20,  # Title font size
            color='white'  # Title font color
        )
    )
)

fig.write_image("./figures/"+modeltype+"_radarplot.png", scale=2,width=700,height=500)

fig.show()

C:\Users\VISWAMBHAR YASA\AppData\Local\Temp\ipykernel_2392\4007647761.py:13: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

